In [ ]:
# Clone GoodbyeLaver
!git clone https://github.com/kdrkdrkdr/GoodbyeLaver

# Install hent-AI
%cd /content/GoodbyeLaver/hent-AI/
!wget https://github.com/kdrkdrkdr/GoodbyeLaver/releases/download/1.0/4x_FatalPixels_340000_G.pth
!wget https://github.com/kdrkdrkdr/GoodbyeLaver/releases/download/1.0/weights.h5
!pip install --upgrade pip
!pip install -r requirements.txt

# Downgrade tensorflow and keras
!pip uninstall tensorflow -y
!pip uninstall keras -y
!pip install keras==2.3.1
!pip install tensorflow-gpu==1.15
!apt install --allow-change-held-packages libcudnn7=7.4.1.5-1+cuda10.0

!python setup.py install
!add-apt-repository ppa:jon-severinsson/ffmpeg
!apt-get update
!apt-get install ffmpeg

# Install lama_cleaner
%cd /content/GoodbyeLaver/lama_cleaner/
!pip install -r requirements.txt

# Make directory
!mkdir /content/image_original/
!mkdir /content/image_detected/
!mkdir /content/image_uncensored/

# Run lama server
%cd /content/GoodbyeLaver/
!nohup python lama.py --port 5003 &

# Load CUDA
%cd /content/GoodbyeLaver/hent-AI/
from detector import Detector
d = Detector('weights.h5')

In [ ]:
# Run Decensoring!
# Upload the censored image to the "/content/image_original/" folder, and run this cell.
!rm -rf /content/image_detected/*
!rm -rf /content/image_uncensored/*
!rm -rf /content/uncensored_image.zip

# Convert image files to png files.
import os
from PIL import Image
files = os.listdir("/content/image_original/")
for filename in files:
    file_wo_ext, file_ext = os.path.splitext(filename)
    if file_ext.lower() != ".png":
        img_path = f'/content/image_original/{filename}'
        Image.open(img_path).save(f'/content/image_original/{file_wo_ext}.png', 'PNG')
        os.remove(img_path)   

# Detecting bar censored
%cd /content/GoodbyeLaver/hent-AI/
d.run_on_folder(
    input_folder='/content/image_original/',
    output_folder='/content/image_detected/', 
    is_video=False, 
    is_mosaic=False,
    dilation=3
)

# Inpainting image
%cd /content/GoodbyeLaver/
!python inpainting.py

# Download Uncensored Image
!zip -r /content/uncensored_image.zip /content/image_uncensored/
from google.colab import files
files.download("/content/uncensored_image.zip")

In [6]:
# Run this cell if you want to delete all uploaded images.
!rm -rf /content/image_original/*